# More fun with sentiment

A new set of packages to play with

In [1]:
#install.packages("rlang")
#install.packages("tidyverse")
#install.packages("tidytext")
#install.packages("glue")
#install.packages("stringr")
#install.packages("lexicon")
library(tidyverse)
library(tidytext)
library(glue)
library(stringr)
library(lexicon)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘glue’

The following object is masked from ‘package:dplyr’:

    collapse



In [2]:
bob <- glue(read_file("data/cnn.txt"))

In [11]:
tokens <- data_frame(text = bob)  %>% unnest_tokens(word, text)
class(tokens)

[1] "tbl_df"     "tbl"        "data.frame"

In [12]:
head(tokens,9)

word
justice
department
special
counsel
robert
mueller's
one
two
punch


In [13]:
t1 <- tokens %>% inner_join(get_sentiments("bing"))

Joining, by = "word"


In [14]:
head(t1,4)

word,sentiment
punch,negative
forceful,negative
trump,positive
trump,positive


In [16]:
t2 <- count(t1,sentiment)
head(t2)

sentiment,n
negative,32
positive,19


In [19]:
str(get_sentiments("bing"))

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	6788 obs. of  2 variables:
 $ word     : chr  "2-faced" "2-faces" "a+" "abnormal" ...
 $ sentiment: chr  "negative" "negative" "positive" "negative" ...


In [22]:
t3 <- spread(t2,sentiment,n,fill=0) %>% mutate(sentiment=positive-negative)
t3

negative,positive,sentiment
32,19,-13


In [10]:
t3$sentiment/(t3$negative+t3$positive)

[1] -0.254902

In [11]:
head(get_sentiments("bing"))
sue <- get_sentiments("bing")
class(sue)

word,sentiment
2-faced,negative
2-faces,negative
a+,positive
abnormal,negative
abolish,negative
abominable,negative


[1] "tbl_df"     "tbl"        "data.frame"

In [12]:
class(stop_words)
head(stop_words)

[1] "tbl_df"     "tbl"        "data.frame"

word,lexicon
a,SMART
a's,SMART
able,SMART
about,SMART
above,SMART
according,SMART


In [24]:
nrc <- get_sentiments("nrc")
head(nrc,9)
length(unlist(nrc[,1]))

word,sentiment
abacus,trust
abandon,fear
abandon,negative
abandon,sadness
abandoned,anger
abandoned,fear
abandoned,negative
abandoned,sadness
abandonment,anger


[1] 6468

In [14]:
#help(data.frame)

In [15]:
silly <- data.frame(word=c("monday","there","message","message"),
                    sentiment=c("sad","silly","devious","sad"),
                   stringsAsFactors=FALSE) # This part VERY important

In [16]:
class(silly)
head(silly,3)

[1] "data.frame"

word,sentiment
monday,sad
there,silly
message,devious


In [17]:
#help(inner_join)

In [18]:
bob<-inner_join(tokens,silly)

Joining, by = "word"


In [19]:
count(inner_join(tokens,silly),sentiment)

Joining, by = "word"


sentiment,n
devious,1
sad,7
silly,1


In [20]:
bobC <- spread(count(bob,sentiment),sentiment,n,fill=0)
bobC

devious,sad,silly
1,7,1


In [21]:
#tokens
gianni <- count(tokens,word)
gianni <- gianni[,-2]
gianni$sentiment <- "drop"
head(gianni,30)
#write.csv(gianni,file="data/mywords.csv")

word,sentiment
12,drop
13,drop
17,drop
2001,drop
2016,drop
27,drop
30,drop
31,drop
5,drop
a,drop


In [22]:
help(read.csv)

In [23]:
tt <-read.csv("data/mywords.csv",stringsAsFactors=FALSE)

In [24]:
tt1 <- tt[,-1]
tt1 <- tt1[tt1$sentiment != "drop",]
head(tt1)

,word,sentiment
7,account,rich
8,accounts,rich
9,additional,rich
10,adviser,rich
11,advisers,rich
14,agent,rich


In [25]:
tx1 <- inner_join(tokens,tt1)
ct1 <- count(tx1,sentiment)
ct2 <- spread(ct1,sentiment,n)
ct2

Joining, by = "word"


poor,rich
39,43


In [26]:
(ct2$rich-ct2$poor)/(ct2$rich+ct2$poor)

[1] 0.04878049